# Earnings Calendar Scraper

This code will go to NASDAQ website and access the ticker symbols that will be releasing earnings reports after market close. The program will then store the symbols in a dataframe which will be saved in the local drive. The program will then use the ticker symbols to get historical pricing for the past n years in the notebook "nasdaq_stock_prices". Additionally, another program "invesing_com_scrape" will retrieve the historical earnings (9 years back if applicable). Finally, "stock_analysis" will analyze.

In [1]:
#import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import datetime
import time
import pandas as pd
import os

In [2]:
#create a function that goes to the earnings page
def earnings_calendar(nasdaq_earnings_page):
    
    #navigating to website section
    try:
        #Set up the webdriver
        global driver
        
        driver = webdriver.Chrome()       

        # Navigate to the webpage
        driver.get(nasdaq_earnings_page)
        
        #sleep
        time.sleep(20)

        
    except:
        "Could not get to the webpage"

In [3]:
#function to get the time of the earnings report since we are interested in earnings released after market close
def time_of_earnings():
    
    global driver

    #attempt to get the elements that pertain to the image icons found in the table. We are interested in the one with the moon
    #we can accomplish this by clicking on the "Time" symbol which automatically sorts to the earnings realesed at market close
    
    try:
        #single find element because there is only one time icon on the page
        time_icon = driver.find_element(By.XPATH, "//button[contains(@class, 'market-calendar-table__columnheader-content')]")
    
    except:
        
        print('Time Icon location is invalid.')
                                        
    #attempt to click the button
    try:
        
        #click the button to sort the table by earnings that are released at market close
        time_icon.click()
    
    #catch error
    except:
        print('There was an error clicking the Time Icon button.')
        

In [4]:
#fucnction to find the table and parse data
def earnings_table_finder():
    
    global driver

    #attempt to find the earnings calendar
    try:
        #using find_element because the code will retrieve the first instance which applies here
        earnings_calendar_elements = driver.find_element(By.XPATH, '//table[@class="market-calendar-table__table"]')
        
        #store the text in a variable
        earnings_raw_data = earnings_calendar_elements.text

    
    #catch the error
    except:
        print('Earnings calendar location invalid')


    # return ticker_symbols
    return earnings_calendar_elements

In [5]:
#function takes in the table web page element and prints out the data row by row
def table_parser(table_web_element):

    #ticker symbol list
    ticker_symbols = []

    # Find all the rows in the table using a CSS selector
    rows = table_web_element.find_elements(By.CSS_SELECTOR, "tr")
    
    # Loop through each row and extract the data
    for row in rows:
        
        # Find all the cells in the row using a CSS selector
        cells = row.find_elements(By.CSS_SELECTOR, "td")

        # Loop through each cell and extract the text
        for cell in cells:
            
            # Extract the text from the cell
            row_data = cell.text

            # print(row_data)
            ticker_symbols.append(row_data)
    
    
    #alter the list so that you get every 8th element of the list (ticker symbol)
    ticker_symbols = ticker_symbols[::8]    
    
    #return the tickers
    return ticker_symbols

# Mechanism that pulls ticker symbols from NASDAQ

In [6]:
#main mechanism that pulls the data of interest

#nasdaq url
nasdaq_url = 'https://www.nasdaq.com/market-activity/earnings'

#function that navigates to the NASDAQ earnings calendar page, by default it takes you to today's stocks.
earnings_calendar(nasdaq_url)

#Filters by the companies that release earnings after market close
icons_trial = time_of_earnings()          

#retrieves the ticker symbols that will release earnings after market close
earnings_table_element = earnings_table_finder()

#input the table element into the table parser function to get tickers
ticker_symbols = table_parser(earnings_table_element)

#display the total number of tickers that are releasing earnings at market close
print('Today\'s Total Tickers', len(ticker_symbols))

#store today's date
today = datetime.date.today()

#create dataframe to send to csv
ticker_df = pd.DataFrame({'tickers':ticker_symbols})
ticker_df.to_csv(f'tickers_for_{today}.csv')

#display the first 5 tickers
print(ticker_symbols[:5])

Today's Total Tickers 50
['PALT', 'TNON', 'CPIX', 'LSTA', 'SALM']
